## Cost functions

# 成本

在本课中，我们将学习如何评估*成本函数*：

- 首先，我们将了解[Qiskit Runtime 原语](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html)
- 定义*成本函数*$C(\vec\theta)$。这是一个特定于问题的函数，定义优化器最小化（或最大化）问题的目标
- 使用 Qiskit Runtime 原语定义测量策略，以优化速度与精度

&nbsp;

![成本函数工作流程](images/cost_function_workflow.png)

## Primitives

所有物理系统，无论是经典的还是量子的，都可以以不同的状态存在。例如，道路上的汽车可以具有表征其状态的特定质量、位置、速度或加速度。同样，量子系统也可以有不同的配置或状态，但它们与经典系统的不同之处在于我们处理测量和状态演化的方式。这导致了量子力学独有的独特性质，例如*叠加*和*纠缠*。就像我们可以使用速度或加速度等物理属性来描述汽车的状态一样，我们也可以使用可观*测量*（数学对象）来描述量子系统的状态。

In quantum mechanics, states are represented by normalized complex column vectors, or *kets* ($|\psi\rangle$), and observables are hermitian linear operators ($\hat{H}=\hat{H}^{\dagger}$) that act on the kets. An eigenvector ($|\lambda\rangle$) of an observable is known as an *eigenstate*. Measuring an observable for one of its eigenstates ($|\lambda\rangle$) will give us the corresponding eigenvalue ($\lambda$) as readout.

如果您想知道如何测量量子系统以及可以测量什么，Qiskit 提供了两个可以提供帮助的[原语](gloss:primitives)：

- `Sampler` ：给定一个量子状态 $|\psi\rangle$，该原语获得每个可能的计算基础状态的概率。
- `Estimator` ：给定一个量子可观测值 $\hat{H}$ 和一个状态 $|\psi\rangle$，该原语计算 $\hat{H}$ 的期望值。


### 采样器原语

给定准备状态 $|\psi\rangle$ 的量子电路， `Sampler`原语根据计算基础计算获得每个可能状态 $|k\rangle$ 的概率。它计算

$$
p_k = |\langle k | \psi \rangle|^2 \quad \forall k \in \mathbb{Z}_2^n \equiv \{0,1,\cdots,2^n-1\},
$$

其中$n$是量子位的数量，$k$是任何可能的输出二进制字符串${0,1}^n$的整数表示（即以$2$为底的整数）。

[Qiskit Runtime 的](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Sampler.html)`Sampler`在量子设备上多次运行电路，在每次运行中执行测量，并根据恢复的位串重建概率分布。它执行的运行（或*射击*）越多，结果就越准确，但这需要更多的时间和量子资源。

然而，由于可能的输出数量随着量子位 $n$（即 $2^n$）的数量呈指数增长，因此为了捕获*密集的*概率分布，镜头数量也需要呈指数增长。因此， `Sampler`只对*稀疏*概率分布有效；其中目标状态 $|\psi\rangle$ 必须可表示为计算基础状态的线性组合，其中项数最多随量子位数呈多项式增长：

$$
|\psi\rangle = \sum^{\text{Poly}(n)}_k w_k |k\rangle.
$$

`Sampler`还可以配置为从电路的一个子部分检索概率，表示全部可能状态的子集。

### 估计器原语

`Estimator`原语计算量子态 $|\psi\rangle$ 的可观测 $\hat{H}$ 的期望值；其中可观测概率可以表示为 $p_\lambda = |\langle\lambda|\psi\rangle|^2$，$|\lambda\rangle$ 是可观测 $\hat{H}$ 的本征态。然后，期望值被定义为状态 $|\psi\rangle$ 测量的所有可能结果 $\lambda$（即可观测值的特征值）的平均值，并按相应的概率加权：

$$
\langle\hat{H}\rangle_\psi := \sum_\lambda p_\lambda \lambda = \langle \psi | \hat{H} | \psi \rangle
$$

然而，计算可观测值的期望值并不总是可能的，因为我们通常不知道它的特征基。 [Qiskit Runtime 的](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Estimator.html)`Estimator`使用复杂的代数过程，通过将可观测量分解为我们确实知道其特征基的其他可观测量的组合来估计真实量子设备的期望值。

简单来说， `Estimator`将任何它不知道如何测量的可观察量分解为更简单、可测量的可观察量，称为[泡利算子](gloss:pauli)。

任何运算符都可以表示为 $4^n$ Pauli 运算符的组合。

$$
\hat{P}_k := 
\sigma_{k_{n-1}}\otimes \cdots \otimes \sigma_{k_0} \quad 
\forall k \in \mathbb{Z}_4^n \equiv \{0,1,\cdots,4^n-1\}, \\
$$

这样

$$
\hat{H} = \sum^{4^n-1}_{k=0} w_k \hat{P}_k
$$

其中$n$是量子位的数量，$k \equiv k_{n-1} \cdots k_0$ for $k_l \in \mathbb{Z}_4 \equiv {0, 1, 2, 3}$ （即整数基数 $4$) 和 $(\sigma_0, \sigma_1, \sigma_2, \sigma_3) := (I, X, Y, Z)$。

执行此分解后， `Estimator`为每个可观测值$\hat{P}_k$（即来自原始电路）导出一个新电路$V_k|\psi\rangle$，以在计算基础上有效地*对*角化泡利可观测值并对其进行测量。我们可以轻松测量泡利可观测量，因为我们提前知道 $V_k$，而其他可观测量通常情况并非如此。

对于每个`{latex} \hat{P}_{k}` ， `Estimator`在量子器件上多次运行相应的电路，在计算基础上测量输出状态，并计算获得每个的概率 $p_{kj}$可能的输出$j$。然后查找每个输出$j$对应的$P_k$的特征值$\lambda_{kj}$，乘以$w_k$，并将所有结果相加，得到可观察的$\hat{H的期望值}$ 对于给定状态 $|\psi\rangle$。

$$
\langle\hat{H}\rangle_\psi = 
\sum_{k=0}^{4^n-1} w_k \sum_{j=0}^{2^n-1}p_{kj} \lambda_{kj},
$$

由于计算 $4^n$ Paulis 的期望值是不切实际的（即指数增长），因此`Estimator`仅当大量 $w_k$ 为零时（即*稀疏*Pauli 分解而不是*密集分解*）才能有效。正式地，我们说，为了*有效地解决*这个计算，非零项的数量最多必须随着量子位 $n$ 的数量以多项式增长：$\hat{H} = \sum^{\text{Poly }(n)}_k w_k \hat{P}_k.$

读者可能会注意到隐含的假设，即概率[采样](gloss:sampling)也需要高效，如`Sampler`所解释的那样，这意味着

$$
\langle\hat{H}\rangle_\psi = 
\sum_{k}^{\text{Poly}(n)} w_k \sum_{j}^{\text{Poly}(n)}p_{kj} \lambda_{kj}.
$$

### 计算期望值的指导示例

我们假设单量子位状态 $|+\rangle := H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$，并且可观察

$$
\begin{aligned}
\hat{H}
& = \begin{pmatrix} 
-1 & 2 \\
2 & -1 \\
\end{pmatrix}\\[1mm]
& = 2X - Z

\end{aligned}
$$

理论期望值如下 $\langle\hat{H}\rangle_+ = \langle+|\hat{H}|+\rangle = 2.$

由于我们不知道如何测量这个可观测量，因此无法直接计算其期望值，需要将其重新表示为 $\langle\hat{H}\rangle_+ = 2\langle X \rangle_+ - \langle Z \rangle_+ $.通过注意到 $\langle+|X|+\rangle = 1$ 和 $\langle+|Z|+\rangle = 0$，可以得出相同的结果。

让我们看看如何直接计算 $\langle X \rangle_+$ 和 $\langle Z \rangle_+$ 。由于$X$和$Z$不交换（即不共享相同的特征基），因此不能同时测量它们，因此我们需要辅助电路：

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# The following code will work for any other initial single-qubit state and observable
original_circuit = QuantumCircuit(1)
original_circuit.h(0)

H = SparsePauliOp(["X", "Z"], [2, -1])

aux_circuits = []
for pauli in H.paulis:
    aux_circ = original_circuit.copy()
    aux_circ.barrier()
    if str(pauli) == "X":
        aux_circ.h(0)
    elif str(pauli) == "Y":
        aux_circ.sdg(0)
        aux_circ.h(0)
    else:
        aux_circ.i(0)
    aux_circ.measure_all()
    aux_circuits.append(aux_circ)


print("Original circuit:")
display(original_circuit.draw("mpl"))
for (circuit, pauli) in zip(aux_circuits, H.paulis):
    print(f"Auxiliary circuit for {str(pauli)}")
    display(circuit.draw("mpl"))

我们现在可以使用`Sampler`手动执行计算并在`Estimator`上检查结果：

In [ ]:
from qiskit.primitives import Sampler, Estimator
from qiskit.circuit.library import IGate, ZGate
import numpy as np


## SAMPLER
sampler = Sampler()
job = sampler.run(aux_circuits)
probability_dists = job.result().quasi_dists

expvals = []
for dist, pauli in zip(probability_dists, H.paulis):
    val = 0
    if str(pauli) == "I":
        Lambda = IGate().to_matrix().real
    else:
        Lambda = ZGate().to_matrix().real
    val += Lambda[0][0] * dist.get(0, 0)
    val += Lambda[1][1] * dist.get(1, 0)
    expvals.append(val)


print("Sampler results:")
for (pauli, expval) in zip(H.paulis, expvals):
    print(f"  >> Expected value of {str(pauli)}: {expval:.5f}")

total_expval = np.sum(H.coeffs * expvals).real
print(f"  >> Total expected value: {total_expval:.5f}")


## ESTIMATOR
observables = [
    *H.paulis,
    H,
]  # Note: run for individual Paulis as well as full observable H

estimator = Estimator()
job = estimator.run([original_circuit] * len(observables), observables)
estimator_expvals = job.result().values

print("Estimator results:")
for (obs, expval) in zip(observables, estimator_expvals):
    if obs is not H:
        print(f"  >> Expected value of {str(obs)}: {expval:.5f}")
    else:
        print(f"  >> Total expected value: {expval:.5f}")

### 数学严谨性（可选）

相对于 $\hat{H}$ 的本征态基表示 $|\psi\rangle$，$|\psi\rangle = \sum_\lambda a_\lambda |\lambda\rangle$，如下：

$$
\begin{aligned}
\langle \psi | \hat{H} | \psi \rangle
& = \bigg(\sum_{\lambda'}a^*_{\lambda'} \langle \lambda'|\bigg) \hat{H} 
  \bigg(\sum_{\lambda} a_\lambda | \lambda\rangle\bigg)\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} 
  \langle \lambda'|\hat{H}| \lambda\rangle\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} \lambda 
\langle \lambda'| \lambda\rangle\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} \lambda 
\cdot \delta_{\lambda, \lambda'}\\[1mm]

& = \sum_\lambda |a_\lambda|^2 \lambda\\[1mm]

& = \sum_\lambda p_\lambda \lambda\\[1mm]

\end{aligned}
$$

由于我们不知道目标可观测量 $\hat{H}$ 的特征值或特征态，因此首先我们需要考虑其对角化。假设 $\hat{H}$ 是[Hermitian](gloss:hermitian) ，则存在酉变换 $V$ 使得 $\hat{H}=V^\dagger \Lambda V,$ 其中 $\Lambda$ 是对角特征值矩阵，所以$\langle j | \拉姆达| k \rangle = 0$ 如果 $j\neq k$，且 $\langle j | \拉姆达| j \rangle = \lambda_j$。

这意味着期望值可以重写为：

$$
\begin{aligned}
\langle\psi|\hat{H}|\psi\rangle
& = \langle\psi|V^\dagger \Lambda V|\psi\rangle\\[1mm]

& = \langle\psi|V^\dagger \bigg(\sum_{j=0}^{2^n-1} |j\rangle 
\langle j|\bigg) \Lambda \bigg(\sum_{k=0}^{2^n-1} |k\rangle \langle k|\bigg) V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1} \sum_{k=0}^{2^n-1}\langle\psi|V^\dagger |j\rangle 
\langle j| \Lambda  |k\rangle \langle k| V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1}\langle\psi|V^\dagger |j\rangle 
\langle j| \Lambda  |j\rangle \langle j| V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1}|\langle j| V|\psi\rangle|^2 \lambda_j\\[1mm]

\end{aligned}
$$

假设如果系统处于状态 $|\phi\rangle = V |\psi\rangle$ 测量 $| 的概率j\rangle$ 为 $p_j = |\langle j|\phi \rangle|^2$，上面的期望值可以表示为：

$$
\langle\psi|\hat{H}|\psi\rangle = 
\sum_{j=0}^{2^n-1} p_j \lambda_j.
$$

需要注意的是，概率取自状态 $V |\psi\rangle$ 而不是 $|\psi\rangle$。这就是为什么矩阵 $V$ 是绝对必要的。

您可能想知道如何获取矩阵 $V$ 和特征值 $\Lambda$。如果您已经有了特征值，那么就不需要使用量子计算机，因为变分算法的目标是找到 $\hat{H}$ 的这些特征值。

幸运的是，有一种解决方法：任何 $2^n \times 2^n$ 矩阵都可以写为 $n$ 泡利矩阵和恒等式的 $4^n$ 张量乘积的线性组合，所有这些都是埃尔米特矩阵和恒等式与已知的 $V$ 和 $\Lambda$ 一致。这就是 Runtime 的`Estimator`通过将任何[Operator](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Operator.html)对象分解为[SparsePauliOp](https://qiskit.org/documentation/stubs/qiskit.quantum_info.SparsePauliOp.html)来内部执行的操作。

以下是可以使用的运算符：

$$
\begin{array}{c|c|c|c}
  \text{Operator} & \sigma & V & \Lambda \\[1mm]
  \hline
  I & \sigma_0 = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} & V_0 = I & \Lambda_0 = I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \\[4mm]

  X & \sigma_1 = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} & V_1 = H =\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix} & \Lambda_1 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \\[4mm]

  Y & \sigma_2 = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} & V_2 = HS^\dagger  =\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}\cdot  \begin{pmatrix} 1 & 0 \\ 0 & -i \end{pmatrix} = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & -i \\ 1 & i \end{pmatrix}\quad & \Lambda_2 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \\[4mm]

  Z & \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} & V_3 = I & \Lambda_3 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
\end{array}
$$

因此，让我们根据 Paulis 和恒等式重写 $\hat{H}$：

$$
\hat{H} = 
\sum_{k_{n-1}=0}^3...
\sum_{k_0=0}^3 w_{k_{n-1}...k_0} 
\sigma_{k_{n-1}}\otimes ... \otimes \sigma_{k_0} = \sum_{k=0}^{4^n-1} w_k \hat{P}_k,
$$

其中 $k = \sum_{l=0}^{n-1} 4^l k_l \equiv k_{n-1}...k_0$ for $k_{n-1},...,k_0\in {0,1,2,3}$（即基础$4$），和`{latex} \hat{P}_{k} := \sigma_{k_{n-1}}\otimes ... \otimes \sigma_{k_0}` ：

$$
\begin{aligned}
\langle\psi|\hat{H}|\psi\rangle
& = \sum_{k=0}^{4^n-1} w_k 
\sum_{j=0}^{2^n-1}|\langle j| V_k|\psi\rangle|^2 \langle j| \Lambda_k |j\rangle \\[1mm]

& = \sum_{k=0}^{4^n-1} w_k \sum_{j=0}^{2^n-1}p_{kj} \lambda_{kj}, \\[1mm]


\end{aligned}
$$

其中 $V_k := V_{k_{n-1}}\otimes ... \otimes V_{k_0}$ 和 $\Lambda_k := \Lambda_{k_{n-1}}\otimes ... \otimes \ Lambda_{k_0}$，这样：$\hat{P_k}=V_k^\dagger \Lambda_k V_k.$

## 成本函数

一般来说，成本函数用于描述问题的目标以及试验状态相对于该目标的执行情况。这个定义可以应用于化学、机器学习、金融、优化等领域的各种例子。

让我们考虑一个寻找系统基态的简单示例。我们的目标是最小化可观测代表能量的期望值（哈密顿 $\hat{\mathcal{H}}$）：

$$
\min_{\vec\theta} \langle\psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle
$$

我们可以使用[*Estimator*原语](https://github.com/qiskit-community/prototype-zne/blob/main/docs/tutorials/0-estimator.ipynb)来评估期望值，并将该值传递给优化器以最小化。如果优化成功，它将返回一组最优参数值 $\vec\theta^ *$，从中我们将能够构造建议的解决方案状态 $|\psi(\vec\theta^* )\rangle$ 和将观察到的期望值计算为 $C(\vec\theta^*)$。

请注意，我们如何只能最小化我们正在考虑的有限状态集的成本函数。这导致我们有两种不同的可能性：

- **我们的 ansatz 没有定义整个搜索空间的解决方案状态**：如果是这种情况，我们的优化器将永远找不到解决方案，我们需要尝试其他可能能够更准确地表示我们的搜索空间的 ansatz。
- **我们的优化器无法找到这个有效的解决方案**：优化可以全局定义和局部定义。我们将在后面的部分中探讨这意味着什么。

总而言之，我们将执行经典的优化循环，但依赖于对量子计算机的成本函数的评估。从这个角度来看，人们可以将优化视为一种纯粹的经典尝试，每次优化器需要评估成本函数时，我们都会调用一些[*黑盒量子预言机*](gloss:oracle)。

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

# Add your token below
service = QiskitRuntimeService(channel="ibm_quantum")

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("XX", 1), ("YY", -3)])

    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values


theta_list = (2 * np.pi * np.random.rand(1, 8)).tolist()
cost_function_vqe(theta_list)

### 映射到非物理系统的示例

最大割（Max-Cut）问题是一个组合优化问题，涉及将图的顶点划分为两个不相交的集合，以使两个集合之间的边数最大化。更正式地说，给定一个无向图 $G=(V,E)$，其中 $V$ 是顶点集，$E$ 是边集，最大割问题要求将顶点划分为两个不相交的子集、$S$ 和 $T$，使得一个端点在 $S$ 且另一个端点在 $T$ 的边数最大化。

我们可以应用 Max-Cut 来解决各种问题，包括：聚类、网络设计、相变等。我们首先创建一个问题图：

In [ ]:
import networkx as nx

n = 4
G = nx.Graph()
G.add_nodes_from(range(n))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
G.add_weighted_edges_from(edge_list)

colors = ["red" for i in range(n)]


def draw_graph(G, colors):
    """Draws the graph with the chose colors"""
    layout = nx.shell_layout(G)
    nx.draw_networkx(G, node_color=colors, pos=layout)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)


draw_graph(G, colors)

该问题可以表示为二元优化问题。对于每个节点 $0 \leq i &lt; n$，其中 $n$ 是图的节点数（在本例中 $n=4$），我们将考虑二进制变量 $x_i$。如果节点 $i$ 是我们将标记为 $1$ 的组之一，则该变量的值将是 $1$；如果它在另一组中，我们将标记为 $0$，则该变量的值为 $0$。我们还将表示从节点 $i$ 到节点 $j$ 的边的权重 $w_{ij}$ （邻接矩阵 $w$ 的元素 $(i,j)$）。因为该图是无向的，$w_{ij}=w_{ji}$。然后我们可以将我们的问题表述为最大化以下成本函数：

$$
\begin{aligned}
C(\vec{x})
& =\sum_{i,j=0}^n w_{ij} x_i(1-x_j)\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i,j=0}^n w_{ij} x_ix_j\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i=0}^n \sum_{j=0}^i 2w_{ij} x_ix_j
\end{aligned}
$$

为了用量子计算机解决这个问题，我们将成本函数表示为可观测值的期望值。然而，Qiskit 承认的可观测量本身由泡利算子组成，其特征值为 $1$ 和 $-1$，而不是 $0$ 和 $1$。这就是为什么我们要对变量进行以下更改：

其中$\vec{x}=(x_0,x_1,\cdots,x_{n-1})$。我们可以使用邻接矩阵 $w$ 轻松访问所有边的权重。这将用于获得我们的成本函数：

$$
z_i = 1-2x_i \rightarrow x_i = \frac{1-z_i}{2}
$$

这意味着：

$$
\begin{array}{lcl} x_i=0 & \rightarrow & z_i=1 \\ x_i=1 & \rightarrow & z_i=-1.\end{array}
$$

所以我们想要最大化的新成本函数是：

$$
\begin{aligned}
C(\vec{z})
& = \sum_{i,j=0}^n w_{ij} \bigg(\frac{1-z_i}{2}\bigg)\bigg(1-\frac{1-z_j}{2}\bigg)\\[1mm]

& = \sum_{i,j=0}^n \frac{w_{ij}}{4} - \sum_{i,j=0}^n \frac{w_{ij}}{4} z_iz_j\\[1mm]

& = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j
\end{aligned}
$$

此外，量子计算机的自然趋势是寻找最小值（通常是最低能量）而不是最大值，因此我们不会最大化 $C(\vec{z})$，而是将其最小化：

$$
-C(\vec{z}) =  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

现在我们有了一个成本函数来最小化其变量的值为 $-1$ 和 $1$，我们可以与 Pauli $Z$ 进行以下类比：

$$
z_i \equiv Z_i = \overbrace{I}^{n-1}\otimes ... \otimes \overbrace{Z}^{i} \otimes ... \otimes \overbrace{I}^{0}
$$

换句话说，变量 $z_i$ 将相当于作用于量子位 $i$ 的 $Z$ 门。而且：

$$
Z_i|x_{n-1}\cdots x_0\rangle = z_i|x_{n-1}\cdots x_0\rangle \rightarrow \langle x_{n-1}\cdots x_0 |Z_i|x_{n-1}\cdots x_0\rangle = z_i
$$

那么我们要考虑的可观察的是：

$$
\hat{H} = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} Z_iZ_j
$$

之后我们必须添加独立项：

$$
\texttt{offset} = - \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

这种转换可以通过[`QuadraticProgram.to_ising()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.QuadraticProgram.to_ising.html)来完成。

In [ ]:
from qiskit_optimization.applications import Maxcut
from qiskit.algorithms.optimizers import COBYLA

w = nx.to_numpy_array(G)

max_cut = Maxcut(w)
quadratic_program = max_cut.to_quadratic_program()
observable, offset = quadratic_program.to_ising()


def cost_function_max_cut_vqe(theta):

    ansatz = TwoLocal(
        observable.num_qubits, "rx", reps=1
    )

    backend = service.backend("ibmq_qasm_simulator")

    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values


optimizer = COBYLA()

initial_theta = np.ones(8)

optimizer_result = optimizer.minimize(fun=cost_function_max_cut_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

我们将在应用程序中重新审视这个示例，以探索如何利用优化器来迭代搜索空间。一般来说，这包括：

- 利用优化器找到最佳参数
- 将最优参数绑定到 ansatz 以查找特征值
- 将特征值转化为我们的问题定义

## 测量策略：速度与精度

如前所述，我们使用有噪声的量子计算机作为*黑盒预言机*，其中噪声会使检索到的值变得不确定，从而导致随机波动，这反过来又会损害（甚至完全阻止）某些优化器的收敛性提议的解决方案。这是我们在逐步实现量子优势时必须解决的一个普遍问题：

![优势](images/cost_function_path_to_quantum_advantage.png)

我们可以使用 Qiskit Runtime Primitive 的错误抑制和错误缓解选项来解决噪声问题并最大限度地提高当今量子计算机的效用。

### 错误抑制

[错误抑制](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-suppression.html)是指在编译过程中用于优化和转换电路以最大限度地减少错误的技术。这是一种基本的错误处理技术，通常会导致整个运行时[产生](gloss:overhead)一些经典的预处理开销。开销包括通过以下方式在量子硬件上运行的转译电路：

- 使用量子系统上可用的本机门来表达电路
- 将虚拟量子位映射到物理量子位
- 根据连接要求添加 SWAP
- 优化 1Q 和 2Q 门
- 向空闲量子位添加动态解耦，以防止退相干的影响。

基元允许通过设置`optimization_level`选项并选择高级转译选项来使用[错误抑制技术](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-suppression.html)。在后面的课程中，我们将深入研究不同的电路构造方法来改善结果，但对于大多数情况，我们建议设置`optimization_level=3` 。

### 减少错误

[错误缓解](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html)是指允许用户通过对执行时的器件噪声进行建模来减少电路错误的技术。通常，这会导致与模型训练相关的量子预处理开销和经典后处理开销，以通过使用生成的模型来减少原始结果中的错误。

Qiskit 运行时原语的`resilience_level`选项指定针对错误构建的恢复能力。更高的级别会产生更准确的结果，但由于量子采样开销而导致处理时间更长。将错误缓解应用于原始查询时，弹性级别可用于配置成本和准确性之间的权衡。

在实施任何错误缓解技术时，我们期望结果中的[偏差](gloss:bias)相对于之前未缓解的偏差有所减少。在某些情况下，偏见甚至可能消失。然而，这是有代价的。当我们减少估计数量的偏差时，统计变异性将会增加（即方差），我们可以通过在采样过程中进一步增加每个电路的射击次数来解释这一点。这将带来超出减少偏差所需的开销，因此默认情况下不会这样做。我们可以通过调整 options.executions.shots 中每个电路的镜头数量来轻松选择此行为，如下例所示。

![偏差方差权衡](images/cost_function_bias_variance_trade_off.png)

在本课程中，我们将在高层次上探索这些错误缓解模型，以说明 Qiskit 运行时基元无需完整实现细节即可执行的错误缓解。

### 旋转读出误差消除 (T-REx)

旋转读出误差消除（T-REx）使用一种称为泡利旋转的技术来减少量子测量过程中引入的噪声。该技术假设没有特定形式的噪声，这使得它非常通用且有效。

整体工作流程：

1. 通过随机位翻转采集零状态的数据（测量前的 Pauli X）
2. 通过随机位翻转获取所需（噪声）状态的数据（测量前的 Pauli X）
3. 计算每个数据集的特殊函数，然后除以。

&nbsp;

![TRE-X](images/cost_function_trex_data_collection.png)

我们可以使用`options.resilience_level = 1`进行设置，如下面的示例所示。

### 零噪声外推

零噪声外推 (ZNE) 的工作原理是，首先放大准备所需量子态的电路中的噪声，获得几种不同噪声级别的测量结果，然后使用这些测量结果推断无噪声结果。

整体工作流程：

1. 针对多个噪声因素放大电路噪声
2. 运行每个噪声放大电路
3. 推断回零噪声极限

&nbsp;

![零点NE](images/cost_function_zne_stages.png)

我们可以使用`options.resilience_level = 2`进行设置。我们可以通过探索各种`noise_factors` 、 `noise_amplifiers`和`extrapolators`来进一步优化这一点，但这超出了本课程的范围。我们鼓励您尝试使用[此处描述的这些选项](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html#advanced-resilience-options)。

### 概率误差消除

一组电路的概率误差消除 (PEC) 样本，平均模拟噪声反相通道以消除所需计算中的噪声。这个过程有点像降噪耳机的工作原理，并且会产生很好的效果。然而，它不像其他方法那么通用，并且采样开销是指数级的。

整体工作流程：

![聚氯乙烯](images/cost_function_pec_layers.png)

第 1 步：泡利旋转

![PEC旋转](images/cost_function_pec_pauli_twirling.png)

第 2 步：重复图层并学习噪声

![PEC学习](images/cost_function_pec_learn_layer.png)

步骤 3：导出保真度（每个噪声通道的误差）

![PEC富达](images/cost_function_pec_curve_fitting.png)

每种方法都有自己的相关开销：所需量子计算数量（时间）和结果准确性之间的权衡：

$$
\begin{array}{c|c|c|c}
  \text{Methods} & R=1 \text{, T-REx} & R=2 \text{, ZNE} & R=3 \text{, PEC} \\[1mm]
  \hline
  \text{Assumptions} & \text{None} & \text{Ability to scale noise} & \text{Full knowledge of noise} \\[1mm]
  \text{Qubit overhead} & 1 & 1 & 1 \\[1mm]
  \text{Sampling overhead} & 2 & N_{\text{noise-factors}} & \mathcal{O}(e^{\lambda N_{layers}}) \\[1mm]
  \text{Bias} & 0 & \mathcal{O}(\lambda^{N_{\text{noise-factors}}}) & 0 \\[1mm]
\end{array}
$$

### 使用 Qiskit Runtime 的缓解和抑制选项

以下是如何在 Qiskit Runtime 中使用错误缓解和抑制时计算期望值。此过程在整个优化循环中多次发生，但我们保持示例简单以演示如何配置错误缓解和抑制。

In [ ]:
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

theta = Parameter("theta")

qc = QuantumCircuit(2)
qc.x(1)
qc.h(0)
qc.cp(theta, 0, 1)
qc.h(0)
observables = SparsePauliOp.from_list([("ZZ", 1)])

qc.draw("mpl")

In [ ]:
## Setup phases
import numpy as np

phases = np.linspace(0, 2 * np.pi, 50)

# phases need to be expressed as a list of lists in order to work
individual_phases = [[phase] for phase in phases]

In [ ]:
## Create noise model
from qiskit.providers.fake_provider import FakeManila
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import Options


# Make a noise model
fake_backend = FakeManila()
noise_model = NoiseModel.from_backend(fake_backend)

# Set options to include the noise model
options = Options()
options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.configuration().basis_gates,
    "coupling_map": fake_backend.configuration().coupling_map,
    "seed_simulator": 42,
}

In [ ]:
from qiskit_ibm_runtime import Session, QiskitRuntimeService, Estimator

# Add your token below
service = QiskitRuntimeService(channel="ibm_quantum")

noisy_exp_values = []
exp_values_with_em_es = []

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    # generate noisy results
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    noisy_exp_values = result.values

    options.optimization_level = 2
    options.resilience_level = 1

    # leverage mitigation and suppression
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    exp_values_with_em_es = result.values

    session.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(phases, noisy_exp_values, "o", label="Noisy")
plt.plot(phases, exp_values_with_em_es, "o", label="Mitigation + Suppression")
plt.plot(phases, 2 * np.sin(phases / 2) ** 2 - 1, label="Theory")
plt.xlabel("Phase")
plt.ylabel("Expectation")
plt.legend()
plt.show()

通过本课程，您学习了如何创建成本函数：

- 创建成本函数
- 如何利用[Qiskit 运行时原语](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html)来减轻和抑制噪声
- 如何定义测量策略以优化速度与精度

这是我们的高级变分工作负载：

![成本函数电路](images/cost_function_circuit.png)

我们的成本函数在优化循环的每次迭代期间运行。下一课将探讨经典优化器如何使用我们的成本函数评估来选择新参数。